# Import Ports
This example shows how to import ports. In this example, we are going to
- Download a example board
- Create a configuration file
  - Add a circuit port between two nets
  - Add a circuit port between two pins
  - Add a circuit port between two pin groups
  - Add a circuit port between two coordinates
  - Add a coax port
  - Add a port reference to the neareast pin
  - Add distributed ports
- Import the configuration file

## Import the required packages

In [1]:
import json
from pathlib import Path
import tempfile

from pyaedt.downloads import download_file

from pyedb import Edb

AEDT_VERSION = "2024.1"
NG_MODE = False


Download the example PCB data.

In [2]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", destination=temp_folder.name)

## Load example layout

In [3]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

PyAEDT INFO: Logger is initialized in EDB.


PyAEDT INFO: legacy v0.20.0


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database ANSYS-HSD_V1.aedb Opened in 2024.1


PyAEDT INFO: Cell main Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


## Create an empty dictionary to host all configurations

In [4]:
cfg = dict()

## Add a circuit port between two nets

Keywords

- **name**. Name of the port.
- **Reference_designator**. Reference designator of the component.
- **type**. Type of the port. Supported types are 'circuit', 'coax'
- **positve_terminal**. Positive terminal of the port. Supported types are 'net', 'pin', 'pin_group', 'coordinates'
- **negative_terminal**. Negative terminal of the port. Supported types are 'net', 'pin', 'pin_group', 'coordinates',
'nearest_pin'

In [5]:
port_1 = {
    "name": "port_1",
    "reference_designator": "X1",
    "type": "circuit",
    "positive_terminal": {"net": "PCIe_Gen4_TX2_N"},
    "negative_terminal": {"net": "GND"},
}

## Add a circuit port between two pins

In [6]:
port_2 = {
    "name": "port_2",
    "reference_designator": "C375",
    "type": "circuit",
    "positive_terminal": {"pin": "1"},
    "negative_terminal": {"pin": "2"},
}

## Add a circuit port between two pin groups

In [7]:
pin_groups = [
    {"name": "U9_5V_1", "reference_designator": "U9", "pins": ["32", "33"]},
    {"name": "U9_GND", "reference_designator": "U9", "net": "GND"},
]

In [8]:
port_3 = {
    "name": "port_3",
    "type": "circuit",
    "positive_terminal": {"pin_group": "U9_5V_1"},
    "negative_terminal": {"pin_group": "U9_GND"},
}

## Add a circuit port between two coordinates

Keywords

- **layer**. Layer on which the terminal is placed
- **point**. XY coordinate the terminal is placed
- **net**. Name of the net the terminal is placed on

In [9]:
port_4 = {
    "name": "port_4",
    "positive_terminal": {"coordinates": {"layer": "1_Top", "point": ["104mm", "37mm"], "net": "AVCC_1V3"}},
    "negative_terminal": {"coordinates": {"layer": "Inner6(GND2)", "point": ["104mm", "37mm"], "net": "GND"}},
}

## Add a coax port

In [10]:
port_5 = {"name": "port_5", "reference_designator": "U1", "type": "coax", "positive_terminal": {"pin": "AM17"}}

## Add a port reference to the neareast pin

Keywords

- **reference_net**. Name of the reference net
- **search_radius**. Reference pin search radius in meter

In [11]:
port_6 = {
    "name": "port_6",
    "reference_designator": "U15",
    "type": "circuit",
    "positive_terminal": {"pin": "D7"},
    "negative_terminal": {"nearest_pin": {"reference_net": "GND", "search_radius": 5e-3}},
}

## Add distributed ports

Keywords

- **distributed**. Whether to create distributed ports. When set to True, ports are created per pin

In [12]:
ports_distributed = {
    "name": "ports_d",
    "reference_designator": "U7",
    "type": "circuit",
    "distributed": True,
    "positive_terminal": {"net": "VDD_DDR"},
    "negative_terminal": {"net": "GND"},
}

## Add setups in configuration

In [13]:
cfg["pin_groups"] = pin_groups
cfg["ports"] = [port_1, port_2, port_3, port_4, port_5, port_6, ports_distributed]

## Write configuration into as json file

In [14]:
file_json = Path(temp_folder.name) / "edb_configuration.json"
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

## Import configuration into example layout

In [15]:
edbapp.configuration.load(config_file=file_json)
edbapp.configuration.run()

True

## Review

In [16]:
edbapp.ports

{'port_1': <pyedb.dotnet.edb_core.edb_data.ports.CircuitPort at 0x1f0c52a6a00>,
 'port_2': <pyedb.dotnet.edb_core.edb_data.ports.CircuitPort at 0x1f0c52a6e80>,
 'port_3': <pyedb.dotnet.edb_core.edb_data.ports.CircuitPort at 0x1f0c52a80c0>,
 'port_4': <pyedb.dotnet.edb_core.edb_data.ports.GapPort at 0x1f0c52a82c0>,
 'port_5': <pyedb.dotnet.edb_core.edb_data.ports.CoaxPort at 0x1f0c52a9580>,
 'port_6': <pyedb.dotnet.edb_core.edb_data.ports.CircuitPort at 0x1f0c52b1400>,
 'U7_VDD_DDR_T9': <pyedb.dotnet.edb_core.edb_data.ports.CircuitPort at 0x1f0c52b3b40>,
 'U7_VDD_DDR_R1': <pyedb.dotnet.edb_core.edb_data.ports.CircuitPort at 0x1f0c52b3d40>,
 'U7_VDD_DDR_L9': <pyedb.dotnet.edb_core.edb_data.ports.CircuitPort at 0x1f0c52b3fc0>,
 'U7_VDD_DDR_L1': <pyedb.dotnet.edb_core.edb_data.ports.CircuitPort at 0x1f0c5294940>,
 'U7_VDD_DDR_J9': <pyedb.dotnet.edb_core.edb_data.ports.CircuitPort at 0x1f0c5295600>,
 'U7_VDD_DDR_J8': <pyedb.dotnet.edb_core.edb_data.ports.CircuitPort at 0x1f0c52964c0>,
 'U7_

## Save EDB

In [17]:
edbapp.save()
print(f"EDB is saved to {edbapp.edbpath}")

EDB is saved to C:\Users\ansys\AppData\Local\Temp\tmp2zok2jju.ansys\edb/ANSYS-HSD_V1.aedb


## Close EDB

In [18]:
edbapp.close()

True